In [1]:
%matplotlib inline
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.utils as vutils
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils import * 
import eval_train as ev
#from fcn8 import FCN8_VGG16

In [2]:
dataroot = '/data/voc2012'
SAVE_DIR = '/data/model'

In [3]:
batch_size = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device :",device)

device : cuda:0


## Load model

In [4]:
! ls /data/model

SVM_transfer_30_voc.pt		     dlv3_voc_sbd_semisup_g0.pt
cnn_classif_transfer_30_voc.pt	     dlv3_voc_sbd_semisup_g05.pt
deeplabv3_10_autosup_voc.pt	     dlv3_voc_sbd_semisup_g07.pt
deeplabv3_30_semisup_g10_l2_voc.pt   dlv3_voc_sbd_semisup_g1.pt
deeplabv3_30_semisup_g50_alt_voc.pt  fcn16.pt
deeplabv3_30_semisup_voc.pt	     fcn16_bn.pt
deeplabv3_30_voc.pt		     fcn32.pt
deeplabv3_50_voc.pt		     fcn32_bn.pt
deeplabv3_autosupervised_voc07.pt    fcn8.pt
deeplabv3_unsupervised_voc.pt	     fcn8_bn.pt
deeplabv3_voc.pt		     fcn_voc_sbd_100.pt
dlv3_voc_sbd_100.pt		     fcn_voc_sbd_30.pt
dlv3_voc_sbd_30.pt		     iou
dlv3_voc_sbd_rotate_30.pt	     loss


In [5]:
def load_model(file=None,fcn=False):
    if file is None:
        if fcn is False:
            model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
        else:
            model = torchvision.models.segmentation.fcn_resnet101(pretrained=True)
    else:
        model = torch.load(os.path.join(SAVE_DIR,file))
    return model

In [6]:
model = load_model(fcn=True)
model.to(device)

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [7]:
d_iou = ev.eval_model_all_angle(model,batch_size=batch_size,device=device,num_classes=21)

Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar


In [10]:
for k in d_iou.keys():
    print('Scores for datasets rotate by',k,'degrees:')
    print('   mIoU',d_iou[k]['mIoU'],'Accuracy',d_iou[k]['Accuracy'],'CE Loss',d_iou[k]['CE Loss'])

Scores for datasets rotate by 10 degrees:
   mIoU 0.6176631686156961 Accuracy 0.9079374535694801 CE Loss 0.2826683868254291
Scores for datasets rotate by 20 degrees:
   mIoU 0.58010961681115 Accuracy 0.9023821220969458 CE Loss 0.3131884036908984
Scores for datasets rotate by 30 degrees:
   mIoU 0.5384002524554458 Accuracy 0.8950473091967887 CE Loss 0.3506411607300103
Scores for datasets rotate by 330 degrees:
   mIoU 0.5290363909991177 Accuracy 0.8924972024565656 CE Loss 0.35447631820617276
Scores for datasets rotate by 340 degrees:
   mIoU 0.5673107024754308 Accuracy 0.8993408264230601 CE Loss 0.3201805706675183
Scores for datasets rotate by 350 degrees:
   mIoU 0.6112551025453532 Accuracy 0.9069338736983285 CE Loss 0.28626932653748244
